In [50]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

import pandas as pd
import numpy as np

import random
import re
import time
import traceback

- 006751 전체대학
- 126897 KU융합과학기술원
- 127662 KU혁신공유대학
- 105271 건축대학
- 105541 경영대학
- 103041 공과대학
- 100251 교무처
- 127772 국제대학
- 126940 국제처
- 122046 글로벌융합대학
- 126843 대학교육혁신원
- 102761 문과대학
- 127425 부동산과학원
- 104951 사범대학
- 127119 사회과학대학
- 103781 상경대학
- 126841 상허교양대학
- 126896 상허생명과학대학
- 104351 생명환경과학대학
- 105061 수의과대학
- 122045 예술디자인대학
- 105121 이과대학
- 127304 혁신공유대학(글로컬)

## 컬럼 정의
- 연도, 단과대학, 과목번호, 원어여부, 비고, 수강신청유의사항, 평가비율, 교재, 과제내용, 과제수, 주별강의계획서, 중간고사, 기말고사
- 평가비율, 교재, 과제내용, 주별강의계획서, 중간고사, 기말고사 컬럼은 딕셔너리 형태로 데이터가 저장되어 있음
- 교재, 과제내용, 주별강의계획서 컬럼은 과목별 강의계획서 해당 표의 내용을 전부 가져온 것으로, 단순 참고용 

In [52]:
# 초기값
syllabus_list = []  # 각 강의계획서 내용이 데이터프레임 형태로 차례로 저장되는 빈 리스트 정의
debug_trial = 0     # 파싱에 실패하는 횟수

# ================================================================================================

# 크롬 옵션 설정
options = Options()
options.add_argument("--start-maximized")  # 창 최대화
options.add_argument("--incognito")  # 시크릿 모드로 실행
options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화

# 크롬 드라이버 사용
driver = webdriver.Chrome(options=options)
driver.get('https://sugang.konkuk.ac.kr/sugang/jsp/search/searchMainOuter.jsp')

# 메인 창 핸들 저장
main_window_handle = driver.current_window_handle

# 드롭다운 로딩 대기
time.sleep(2)

# 연도 반복
for year in ['2022', '2023', '2024']:  # 텍스트로 지정해야 합니다.
    print(f'Current Year: {year}')

    # 단과대학 번호 목록 (자기설계전공, 현장실습, 학부논문작성연습, 실감미디어, 글로컬 제외)
    univ_dict = {'126897': 'KU융합과학기술원', '105271': '건축대학', '105541': '경영대학', '103041': '공과대학',
                 '102761': '문과대학', '127425': '부동산과학원', '104951': '사범대학', '127119': '사회과학대학',
                 '126896': '상허생명과학대학', '105061': '수의과대학', '122045': '예술디자인대학', '105121': '이과대학'}

    # 드롭다운 - 연도
    select_year = driver.find_element(By.NAME, 'pYear')
    Select(select_year).select_by_value(year)

    # 적응 시간
    time.sleep(np.round(random.uniform(0.5, 1), 3))

    # 드롭다운 - 학기 (2학기만 확인)
    select_semester = driver.find_element(By.NAME, 'pTerm')
    Select(select_semester).select_by_value('B01012')

    # 적응 시간
    time.sleep(np.round(random.uniform(0.5, 1), 3))

    # 2022, 2023년에는 글로벌융합대학 추가, 2024년에는 국제대학 추가
    if year != '2024':
        univ_dict['122046'] = '글로벌융합대학'
    else:
        univ_dict['127772'] = '국제대학'

    # 단과대 반복
    for univ_idx, univ_number in enumerate(univ_dict.keys(), 1):
        print(f'  {univ_idx}. Collecting {univ_dict[univ_number]} . . .')
        
        # 드롭다운 - 대학
        select_univ = driver.find_element(By.NAME, 'pUniv')
        Select(select_univ).select_by_value(univ_number)

        # 글로벌융합대학일 경우 융합인재학부 지정 필요
        if univ_number == '122046':
            select_department = driver.find_element(By.NAME, 'pSustMjCd')  # 드롭다운 - 학과
            Select(select_department).select_by_value('122410')
            time.sleep(0.5)
        
        # 조회 버튼 클릭
        search_button = driver.find_element(By.CLASS_NAME, 'btn-sub')
        search_button.click()
    
        # 적응 시간
        time.sleep(np.round(random.uniform(0.5, 1), 3))
        
        # 과목번호 - 클래스명이 'btn-sm btn-sub'인 모든 버튼 가져오기
        number_buttons = driver.find_elements(By.CLASS_NAME, 'btn-sm.btn-sub')

        # 원어 - aria-describedby="gridLecture_lang_type_nm" 속성을 가진 모든 <td> 요소 찾기
        lang_type = driver.find_elements(By.XPATH, '//td[@aria-describedby="gridLecture_lang_type_nm"]')
        
        # 비고 - aria-describedby="gridLecture_remk" 속성을 가진 모든 <td> 요소 찾기
        lecture_remk = driver.find_elements(By.XPATH, '//td[@aria-describedby="gridLecture_remk"]')
        
        # 적응 시간
        time.sleep(np.round(random.uniform(0.5, 1), 3))
        
        # 각 버튼 클릭 및 팝업 처리
        for row_idx, button in enumerate(number_buttons):
            # 과목번호 추출하기
            onclick_value = button.get_attribute('onclick')
            button_number = re.search(r"\('(\d+)'\)", onclick_value).group(1)  # 정규표현식으로 숫자 부분만 추출
            
            # 클릭할 강의계획서의 내용을 저장할 딕셔너리 정의
            syllabus_dict = {
                '연도': year,
                '단과대학': univ_dict[univ_number],
                '과목번호': button_number,
                '원어여부': lang_type[row_idx].text,
                '비고': lecture_remk[row_idx].text
            }
            
            # 클릭할 버튼이 화면에 보이도록 스크롤
            driver.execute_script("arguments[0].scrollIntoView(true);", button)
            time.sleep(0.5)
            
            # 강의계획서 클릭
            button.click()
            
            # 팝업창 대기 (팝업이 뜨면 창 개수가 증가)
            WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
            
            # 팝업창 핸들 탐색 및 전환
            for handle in driver.window_handles:
                if handle != main_window_handle:
                    driver.switch_to.window(handle)  # 팝업창으로 전환
                    break
            
            # 팝업창에서 원하는 데이터 크롤링
            try:
                # 팝업창에서 텍스트가 모두 뜰 때까지 대기
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'tbl-detail'))
                )
        
                # HTML 소스 가져오기
                page_source = driver.page_source
    
                # BeautifulSoup으로 HTML 파싱
                soup = BeautifulSoup(page_source, 'html.parser')

                # <table class="tbl-detail"> 안의 <td> 태그 텍스트 추출
                tables = soup.find_all('table', class_='tbl-detail')  # 모든 <table class="tbl-detail"> 요소 찾기
                
                # 모든 테이블의 <td> 태그 텍스트를 추출
                for table_idx, table in enumerate(tables, 1):  # 각 테이블에 대해 반복
                    # 현재 테이블의 모든 <td> 태그 가져오기
                    td_elements = table.find_all('td')
                    td_texts = {idx: td.text.strip() for idx, td in enumerate(td_elements, 1)}  # 텍스트만 추출하고 양쪽 공백 제거

                    # 강의계획서 내용이 존재하지 않는 경우 결측치로 처리
                    if len(tables) == 1:
                        break
                        
                    # 일반사항 테이블 - '수강신청 유의사항'만 추츨
                    if table_idx == 1:
                        cleaned_text = td_texts[5].replace("-", "")  # 엑셀 표기상 오류를 막기 위해 텍스트에서 - 제거
                        syllabus_dict['수강신청유의사항'] = cleaned_text.strip()  # 양쪽 공백 제거

                    # 과제비율 테이블 - 비율이 0보다 큰 항목들만 가져오기
                    elif table_idx == 2:
                        rate_dict = {}
    
                        # td_texts를 4개씩 그룹으로 묶어 처리 (최종적으로 항목 이름, 비율만 추출)
                        for i in range(0, len(td_texts), 4):
                            rate = td_texts[i+2]
        
                            # 숫자인지 확인 후, 0보다 크면 딕셔너리에 추가
                            try:
                                rate_value = float(rate)
                                if rate_value > 0:
                                    rate_dict[td_texts[i+1]] = rate_value
                            except ValueError:
                                pass  # 숫자가 아닌 경우 무시
                            
                            # 비율 합이 100이 되면 중단
                            if sum(rate_dict.values()) >= 100:
                                break

                        # 비율이 0보다 큰 항목들만 딕셔너리로 묶어 저장
                        syllabus_dict['과제비율'] = rate_dict
                    
                    # 강의교재 테이블
                    elif table_idx == 3:
                        syllabus_dict['교재'] = td_texts if len(td_texts) > 1 else np.nan
                    
                    # 강의과제 테이블 - 과제내용, 과제 수
                    elif table_idx == 4:
                        syllabus_dict['과제내용'] = td_texts if len(td_texts) > 1 else np.nan
                        syllabus_dict['과제수'] = round(len(td_texts)/3)  # 내용이 없을 경우 0

                    # 주별강의계획서 테이블 - 주별강의계획서 전체내용, 중간고사 행, 기말고사 행
                    else:
                        if len(td_texts) > 1:  # 내용이 없을 경우 입력하지 않음
                            syllabus_dict['주별강의계획서'] = td_texts
                            syllabus_dict['중간고사'] = {
                                '주제': td_texts[44],
                                '강의내용': td_texts[45],
                                '수업유형': td_texts[46],
                                '강의활동': td_texts[47]
                            }
                            syllabus_dict['기말고사'] = {
                                '주제': td_texts[92],
                                '강의내용': td_texts[93],
                                '수업유형': td_texts[94],
                                '강의활동': td_texts[95]
                            }
                
            except Exception: 
                # 오류 메시지 출력
                print(f'{button_number} 과목 - HTML 파싱 실패')
                traceback.print_exc()  # 전체 스택 추적 출력
    
                # 팝업창 스크린샷 촬영
                debug_trial += 1
                driver.save_screenshot(f"debug_screenshot_{debug_trial}.png")
    
            # 강의계획서 업데이트
            syllabus_list.append(pd.DataFrame([syllabus_dict]))
            
            # 팝업창 닫기
            driver.close()
            driver.switch_to.window(main_window_handle)  # 메인 창으로 복귀

# 크롤링 완료
driver.quit()

# 모든 강의계획서를 데이터프레임으로 한번에 병합
result_df = pd.concat(syllabus_list, ignore_index=True)
print("크롤링이 모두 완료되었습니다!")

Current Year: 2022
  1. Collecting KU융합과학기술원 . . .
  2. Collecting 건축대학 . . .
  3. Collecting 경영대학 . . .
  4. Collecting 공과대학 . . .
  5. Collecting 문과대학 . . .
  6. Collecting 부동산과학원 . . .
  7. Collecting 사범대학 . . .
  8. Collecting 사회과학대학 . . .
4088 과목 - HTML 파싱 실패


Traceback (most recent call last):
  File "C:\Users\gustj\AppData\Local\Temp\ipykernel_17188\2441166004.py", line 190, in <module>
    '주제': td_texts[92],
KeyError: 92


  9. Collecting 상허생명과학대학 . . .
  10. Collecting 수의과대학 . . .
  11. Collecting 예술디자인대학 . . .
  12. Collecting 이과대학 . . .
  13. Collecting 글로벌융합대학 . . .
Current Year: 2023
  1. Collecting KU융합과학기술원 . . .
  2. Collecting 건축대학 . . .
  3. Collecting 경영대학 . . .
  4. Collecting 공과대학 . . .
  5. Collecting 문과대학 . . .
  6. Collecting 부동산과학원 . . .
  7. Collecting 사범대학 . . .
  8. Collecting 사회과학대학 . . .
  9. Collecting 상허생명과학대학 . . .
  10. Collecting 수의과대학 . . .
  11. Collecting 예술디자인대학 . . .
  12. Collecting 이과대학 . . .
  13. Collecting 글로벌융합대학 . . .
Current Year: 2024
  1. Collecting KU융합과학기술원 . . .
  2. Collecting 건축대학 . . .
  3. Collecting 경영대학 . . .
  4. Collecting 공과대학 . . .
  5. Collecting 문과대학 . . .
  6. Collecting 부동산과학원 . . .
  7. Collecting 사범대학 . . .
  8. Collecting 사회과학대학 . . .
  9. Collecting 상허생명과학대학 . . .
  10. Collecting 수의과대학 . . .
  11. Collecting 예술디자인대학 . . .
  12. Collecting 이과대학 . . .
  13. Collecting 국제대학 . . .
크롤링이 모두 완료되었습니다!


### 오류 확인
강의계획서 내 주별강의계획서 표는 16개의 행으로 구성되는데, 4088번 과목의 교수님은 행 하나를 임의로 제거하셨음 => 오류 발생.  
해당 부분은 강의계획서를 확인하여 csv 파일 내에서 직접 수정하였음.

In [53]:
result_df.head()

,연도,단과대학,과목번호,원어여부,비고,수강신청유의사항,과제비율,교재,과제내용,과제수,주별강의계획서,중간고사,기말고사
0,2022,KU융합과학기술원,3854,영어,미래에너지공학과 우선수강,없음,"{'출석률': 10.0, '기말고사비율': 80.0, '과제물비율': 10.0}",NaN,"{1: '실험결과를 바탕으로 한 해석', 2: '20191211', 3: ''}",1.0,"{1: '08/29 ~ 09/03', 2: 'Introduction', 3: 'Ba...","{'주제': 'Surface characterization II', '강의내용': ...","{'주제': 'Final exam', '강의내용': 'Final exam', '수업..."
1,2022,KU융합과학기술원,3855,영어,미래에너지공학과만 수강,미래에너지공학과 학생만 수강 가능.\n ( Only Students of dep...,"{'출석률': 20.0, '중간고사비율': 20.0, '기말고사비율': 50.0, ...","{1: '주교재', 2: 'PDF file', 3: '', 4: '', 5: ''}","{1: 'Summary for Literature survey', 2: '20211...",1.0,"{1: '08/29 ~ 09/03', 2: 'Introduction', 3: 'In...","{'주제': 'Mid-term exam', '강의내용': 'Mid-term exam...","{'주제': 'Final exam', '강의내용': 'Final exam', '수업..."
2,2022,KU융합과학기술원,3856,,미래에너지공학과 우선수강,없음,"{'출석률': 10.0, '중간고사비율': 40.0, '기말고사비율': 40.0, ...","{1: '부교재', 2: '기기분석', 3: '정맹준', 4: '드림플러스', 5:...","{1: '나노소재 물성 분석에 대한 최신 기술 동향', 2: '20211206', ...",1.0,"{1: '08/29 ~ 09/03', 2: '강의 개요', 3: '기기분석학 소개'...","{'주제': '나노소재 분석기술', '강의내용': '화학적 특성 분석법', '수업...","{'주제': '-', '강의내용': '-', '수업유형': '', '강의활동': ''}"
3,2022,KU융합과학기술원,3857,영어,미래에너지공학과 우선수강,없음,"{'출석률': 15.0, '중간고사비율': 40.0, '기말고사비율': 40.0, ...","{1: '주교재', 2: '전기화학 2판', 3: '오승모', 4: '자유아카데미'...","{1: '강의 주제 관련 문제 풀이', 2: '20201024', 3: ''}",1.0,"{1: '08/29 ~ 09/03', 2: 'Electrchemistry Funda...","{'주제': 'Electrchemistry Fundamentals', '강의내용':...","{'주제': 'Final Exam & Self-Study', '강의내용': 'Fin..."
4,2022,KU융합과학기술원,3858,,미래에너지공학과 우선수강,"In this class, students should perform a team ...","{'출석률': 10.0, '상호평가': 10.0, '발표': 40.0, '보고서':...",NaN,{1: 'Energy-related materials or device design...,1.0,"{1: '08/29 ~ 09/03', 2: 'Introduction to capst...","{'주제': 'Implementation', '강의내용': 'Implementati...","{'주제': 'Final evaluation', '강의내용': 'Final Eval..."


In [54]:
result_df.tail()

,연도,단과대학,과목번호,원어여부,비고,수강신청유의사항,과제비율,교재,과제내용,과제수,주별강의계획서,중간고사,기말고사
3478,2024,국제대학,4221,,,,"{'출석률': 15.0, '중간고사비율': 35.0, '기말고사비율': 35.0, ...","{1: '주교재', 2: '글로벌 커뮤니케이션', 3: '유세경', 4: '커뮤니케...",NaN,0.0,"{1: '09/02 ~ 09/07', 2: '오리엔테이션', 3: '강사 및 강의 ...","{'주제': '중간고사', '강의내용': '시험 및 과제물 제출', '수업유형': ...","{'주제': '기말 고사', '강의내용': '시험 및 과제물 제출', '수업유형':..."
3479,2024,국제대학,4222,,,,"{'출석률': 20.0, '중간고사비율': 20.0, '기말고사비율': 20.0, ...",NaN,NaN,0.0,"{1: '09/02 ~ 09/07', 2: '수업 소개와 영상 제작 개론 Cour...","{'주제': '중간 평가 Midterm Assignment', '강의내용': '8강...","{'주제': '종합평가 Final Assignment', '강의내용': '16강 ..."
3480,2024,국제대학,4223,,,2학기 글로벌 콘텐츠 세미나는 한국의 방송산업에 대한 기초적 이해 목적으로 합니다....,"{'출석률': 10.0, '중간고사비율': 35.0, '기말고사비율': 35.0, ...",NaN,NaN,0.0,"{1: '09/02 ~ 09/07', 2: '수업소개와 수업방식의 이해', 3: '...","{'주제': '중간고사', '강의내용': '강의실 시험', '수업유형': '', '...","{'주제': '기말고사', '강의내용': '강의실 시험', '수업유형': '', '..."
3481,2024,국제대학,4224,,,각 교과목 중 총 수업시간의 3분의 2 이상을 출석해야만 그 교과목의 시험에 응시할...,"{'출석률': 10.0, '중간고사비율': 35.0, '기말고사비율': 40.0, ...",NaN,NaN,0.0,"{1: '09/02 ~ 09/07', 2: '산업 현황을 이해한다', 3: '오리엔...","{'주제': '중간고사', '강의내용': '중간고사', '수업유형': '과제평가',...","{'주제': '팀 프로젝트 수행', '강의내용': '팀 프로젝트 발표2', '수업유..."
3482,2024,국제대학,4225,,,,"{'출석률': 10.0, '중간고사비율': 30.0, '기말고사비율': 30.0, ...","{1: '주교재', 2: 'Reading Packet', 3: '', 4: '', ...",NaN,0.0,"{1: '09/02 ~ 09/07', 2: '리서치 방법론 과목의 소개', 3: '...","{'주제': '중간고사', '강의내용': '강의실 시험', '수업유형': '', '...","{'주제': '기말고사', '강의내용': '강의실 시험', '수업유형': '', '..."


In [55]:
result_df.shape

(3483, 13)

In [56]:
result_df.to_csv('강의계획서_전선.csv',index=False,encoding='cp949')